<a href="https://colab.research.google.com/github/chokun7788/PoopforAIB/blob/main/Chokun7788.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U fastai scikit-learn

###import

In [ ]:
from fastai.vision.all import *
import timm
from sklearn.metrics import classification_report,f1_score
from fastai.callback.tracker import SaveModelCallback
import matplotlib.pyplot as plt
import numpy as np

###Path

In [ ]:
path = Path('/content/drive/MyDrive/TheRealDataV1')
train_path = path/'train'
valid_path = path/'valid'
test_path = path/'test'

class_names = ['Blood', 'Diarrhea', 'Normal', 'Mucus', 'Yellow', 'Green']
num_classes = len(class_names)
image_size = 224 #ขนาด(กว้างxยาว)
batch_size = 32 #รูปภาพที่ยัดพร้อมกัน

###Augment

In [ ]:
item_tfms = [Resize(image_size,method = ResizeMethod.Squish)]
custom_batch_tfms = aug_transforms(
    do_flip=True, #พลิกแนวนอน
    max_rotate=15.0, #สุ่มหมุน 15องศา
    min_zoom=1.0, #ซูมน้อยสุด
    max_zoom=1.15, #สุ่มซูม
    max_warp=0.15, #สุ่มบิดมุมภาพ
    p_affine=0.75 #สุ่มค.น่าจะเป็น
) + [RandomErasing(p=0.3, sl=0.0, sh=0.25, min_aspect=0.3)] #สุ่มพื้นที่บางส่วนแล้วปรับขนาตาม image_size

###DataLoaders

In [ ]:
dls = ImageDataLoaders.from_folder(
    path,
    train='train',
    valid='valid',
    item_tfms=item_tfms,
    batch_tfms=custom_batch_tfms,
    bs=batch_size,
    seed=42
)
print(f"Classes: {dls.vocab}")
print(f"Number of classes : {dls.c}")

###Model

In [ ]:
model_name = 'convnextv2_tiny.fcmae_ft_in22k_in1k'

In [ ]:
learn = vision_learner(
    dls,
    model_name,
    metrics=[
        accuracy,
        error_rate,
        F1Score(average='macro'),
    ],
    cbs=[
        SaveModelCallback(
            monitor='f1_score',
            fname='best_f1_checkpoint',
            with_opt=True,
            comp=np.greater
        )
    ]
)

###Learning Rate

In [ ]:
# lr_find_results = learn.lr_find(suggest_funcs=(valley, slide))
# print(f"Suggested learning rates: valley={lr_find_results.valley}, slide={lr_find_results.slide}")

###Train Model

In [ ]:
epochs = 30
freeze = 7
lr = 0.0008317637839354575

In [ ]:
learn.fine_tune(
    epochs,
    base_lr = lr,
    freeze_epochs=freeze
)

In [ ]:
learn.recorder.plot_loss()
plt.show()

###Best Model

In [ ]:
best_checkpoint_name = 'best_f1_checkpoint'
checkpoint_path_to_load = learn.path/learn.model_dir/f'{best_checkpoint_name}.pth'

In [ ]:
learn.load(best_checkpoint_name)
print(f"Successfully loaded weights from '{best_checkpoint_name}'.")

###Save Model

In [ ]:
save_model_directory = path/'convnextv2_thev1'
save_model_directory.mkdir(parents=True, exist_ok=True)

In [ ]:
model_export_filename = 'convnextv2_thev1_best.pkl'
model_export_path = save_model_directory/model_export_filename

learn.export(model_export_path)
print(f"Best Full Learner exported to: {model_export_path}")

###Test Set

In [ ]:
best_model_pkl_path = Path('/content/drive/MyDrive/TheRealDataV1/convnextv2_thev1/convnextv2_thev1_best.pkl')
eval_learner = load_learner(best_model_pkl_path)
test_files = get_image_files(test_path).sorted()
test_dl = eval_learner.dls.test_dl(test_files, with_labels=True)

In [ ]:
test_preds_raw, test_targs, test_decoded_preds = eval_learner.get_preds(dl=test_dl, with_decoded=True)
target_names_test = eval_learner.dls.vocab
report_test_dict = classification_report(
  test_targs.numpy(),
  test_decoded_preds.numpy(),
  target_names=target_names_test,
  output_dict=True
)
print(classification_report(
  test_targs.numpy(),
  test_decoded_preds.numpy(),
  target_names=target_names_test
)
)

print(f"Test Set Macro F1-Score (Best Model): {report_test_dict['macro avg']['f1-score']:.4f}")
print(f"Test Set Weighted F1-Score (Best Model): {report_test_dict['weighted avg']['f1-score']:.4f}")

print("Test Set Confusion Matrix (Best Model) ---")
cm_test = ConfusionMatrix(actuals=test_targs.numpy(), preds=test_decoded_preds.numpy())
cm_test.plot(class_names=target_names_test, normalize=True, figsize=(10,10), dpi=70)
plt.title("Test Set Confusion Matrix (Normalized - Best Model)")
plt.show()

In [ ]:
from fastai.interpret import ConfusionMatrix

cm_test = ConfusionMatrix(
  actuals=test_targs.numpy(),      # Label จริง
  preds=test_decoded_preds.numpy() # Label ที่โมเดลทำนาย
)

cm_test.plot(
  class_names=target_names_test, # ชื่อคลาสสำหรับแกน X และ Y
  normalize=False,             # <--- ตั้งเป็น False เพื่อแสดงจำนวนเต็ม
  figsize=(10,10),             # ขนาดของรูปภาพ (ปรับได้ตามความเหมาะสม)
  cmap='Blues',                # ชุดสีที่ใช้ (เช่น 'Blues', 'Greens', 'Reds')
  dpi=70                       # ความละเอียดของรูปภาพ
)
plt.title("Confusion Matrix (Test Set - ConvNeXt Updated)") # <--- ตั้งชื่อหัวเรื่องตามต้องการ
plt.show()